In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'

mpl.rc('font',family='serif',size='16')
fig = plt.figure(1,figsize=(7,5))
#fig = plt.figure(1,figsize=(7, 5)) ## To save figure.

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.) 

<Figure size 504x360 with 0 Axes>

In [2]:
def calc_S(N_tot,Alpha):
    S = np.zeros((N_tot,N_tot))
    for p in range(N_tot):
        for q in range(N_tot):
            factor = np.pi/(Alpha[p]+Alpha[q])
            S[p,q] = factor*np.sqrt(factor)
    return S
def calc_T(Alpha,N_tot):
    T = np.zeros((N_tot,N_tot))
    for i in range(N_tot):
        for j in range(N_tot):
            y = 3.0E0*Alpha[i]*Alpha[j]*np.pi*np.sqrt(np.pi)
            z = (Alpha[i]+Alpha[j])**2*np.sqrt(Alpha[i]+Alpha[j])
            T[i,j] = y/z
    return T
def calc_A(Alpha,N_tot):
    A = np.zeros((N_tot,N_tot))
    for i in range(N_tot):
        for j in range(N_tot):
            A[i,j] = -4.0E0*np.pi/(Alpha[i]+Alpha[j])
    return A

def calc_H(Alpha,N_tot):
    H = np.zeros((N_tot,N_tot))
    T = calc_T(Alpha,N_tot)
    A = calc_A(Alpha,N_tot)
    H = np.add(T,A)
    return H

def calc_Q(Alpha,N_tot):
    Q = np.zeros((N_tot,N_tot,N_tot,N_tot))
    y = 2.0E0 * np.pi*np.pi * np.sqrt(np.pi)
    for r in range(N_tot):
        for s in range(r+1):
            #print("InCalcQ: r, s: ", r, s)
            for t in range(r+1):
                if t < r:
                    MaxU = t+1
                else:
                    MaxU = s+1
                for u in range(MaxU):
                    part_A = Alpha[r] + Alpha[s]
                    part_B = Alpha[t] + Alpha[u]
                    z = part_A*part_B*np.sqrt(part_A+part_B)
                    Q[r,s,t,u] = y/z
                    Q[r,s,u,t] = y/z
                    Q[t,u,r,s] = y/z
                    Q[t,u,s,r] = y/z
    return Q

def calc_G(N_tot,DensityMatrix,Q):
    '''
    Here we used the symmetry, change:
    \sum_{t=1}^{N}\sum_{u=1}^{N} (1/2)D[t,u]*Q[r,s,t,u]
    into:
    \sum_{t=1}^{N}(\sum_{u=1}^{t-1}D[t,u]*Q[r,s,t,u]+(1/2)D[t,t]*Q[r,s,t,t])
    '''
    G = np.zeros((N_tot,N_tot))
    for r in range(N_tot):
        for s in range(r+1):
            G[r,s] = 0.0E0
            for t in range(N_tot):
                for u in range(t):
                    G[r,s] = G[r,s] + DensityMatrix[t,u]*Q[r,s,t,u]
                G[r,s] += 0.5E0*DensityMatrix[t,t]*Q[r,s,t,t]
            G[s,r] = G[r,s]
    return G

def calc_F(N_tot,G,H):
    F = np.zeros((N_tot,N_tot))
    for p in range(N_tot):
        for q in range(N_tot):
            F[p,q] = H[p,q] + G[p,q]
    return F

def Build_DensMat(C,S):
    '''
    D = 2 C.C^{T}/(C^{T}SC), and then normalized to:
    \sum_{i,j} 0.5*D[i,j]*S[i,j].
    Since D is symmetric, we can do it as:
    \sum_{i=1}^{N}(\sum_{j=1}^{i-1} D[i,j]*S[i,j] + 0.5*D[i,i]*S[i,i])
    '''
    DensMat = np.zeros((N_tot,N_tot))
    Norm = 0.0E0
    for r in range(N_tot):
        for s in range(r):
            DensMat[r,s] = 2.0*C[r]*C[s]
            Norm += DensMat[r,s]*S[r,s]
        DensMat[r,r] = 2.0*C[r]*C[r]
        Norm += 0.5*DensMat[r,r]*S[r,r]
        
    Norm = 1.0E0/Norm
    for r in range(N_tot):
        for s in range(r+1):
            DensMat[r,s] = DensMat[r,s]*Norm
            DensMat[s,r] = DensMat[r,s]
    return DensMat

def DiagFock(S,N_tot,F,H, D,V,Vd,Q,i_fact):

    tmp = np.dot(F,V) ### Here used is F.
    H_prime = np.dot(Vd,tmp)
    
    ### Step 5: Calculate the eigenvalue problem.
    EE, C_prime = np.linalg.eigh(H_prime)

    ## print(EE)
    C = np.dot(V,C_prime)
    
    C_use = copy.deepcopy(C[:,0]) ### Ground state only...
    D_new = Build_DensMat(C_use,S) ## Ground state only...

    Ener = 0.0E0
    tmp = 0.0E0

    Ener = EE[0]
    for i in range(N_tot):
        for j in range(i):
            Ener += D_new[i,j]*H[i,j]
        Ener += 0.5E0*D_new[i,i]*H[i,i]
    print("After round %4d Energy: %12.6f" % (i_fact, Ener))
    
    return Ener, D_new

In [3]:
Alpha = np.array([0.298073,1.242567,5.782948,38.47497],dtype=float)
N_tot = 4

C0=np.zeros(N_tot)
for i in range(N_tot):
    C0[i] = 1.0E0
    
S = calc_S(N_tot,Alpha)
D = Build_DensMat(C0,S)
H = calc_H(Alpha,N_tot)
Q = calc_Q(Alpha,N_tot)
G = calc_G(N_tot, D, Q)

small_S, U = np.linalg.eigh(S)

half_S = np.zeros((N_tot,N_tot))
for i in range(N_tot):
    half_S[i,i] = 1.0E0/np.sqrt(np.abs(small_S[i]))
    

V = np.dot(U,half_S)
Vd = np.transpose(V)


In [4]:
OldEner = -1.0E0
Ener    = 0.0E0

ener_traj = []
for i in range(20): 
    OldEner = Ener
    G = calc_G(N_tot,D,Q)
    F = calc_F(N_tot,G,H)
    Ener, D_new = DiagFock(S,N_tot,F,H, D,V,Vd,Q,i)
    ener_traj.append(Ener)
    D = copy.deepcopy(D_new)
    

After round    0 Energy:    -3.033776
After round    1 Energy:    -2.808769
After round    2 Energy:    -2.868018
After round    3 Energy:    -2.851662
After round    4 Energy:    -2.856117
After round    5 Energy:    -2.854899
After round    6 Energy:    -2.855232
After round    7 Energy:    -2.855141
After round    8 Energy:    -2.855166
After round    9 Energy:    -2.855159
After round   10 Energy:    -2.855161
After round   11 Energy:    -2.855160
After round   12 Energy:    -2.855160
After round   13 Energy:    -2.855160
After round   14 Energy:    -2.855160
After round   15 Energy:    -2.855160
After round   16 Energy:    -2.855160
After round   17 Energy:    -2.855160
After round   18 Energy:    -2.855160
After round   19 Energy:    -2.855160


In [5]:
from pyscf import gto, scf

In [10]:
mol = gto.M(
    atom = [['He', (0, 0, 0)]],
    basis = "augccpv6z", unit="bohr")

In [11]:
s=mol.intor("int1e_ovlp_sph")
t=mol.intor("int1e_kin_sph")
vne=mol.intor("int1e_nuc_sph")
print("s, t, vne:")
#print(s)
#print(t)
#print(vne)


Hcore = np.add(t,vne)
nao=mol.nao_nr() ## obtain the number of atomic orbitals.
print("Total number of basis fns:", nao)
print("Shape of overlap, kinetic, vne matrices", s.shape, t.shape, vne.shape)

# Let's now also compute the two-electron integrals
Q=mol.intor("int2e_sph",aosym=1)
Q=np.reshape(Q,[nao,nao,nao,nao])

print ("Number of basis fns=", nao)

s, t, vne:
Total number of basis fns: 80
Shape of overlap, kinetic, vne matrices (80, 80) (80, 80) (80, 80)
Number of basis fns= 80


In [12]:
N_tot = nao

C0=np.zeros(N_tot)
for i in range(N_tot):
    C0[i] = 1.0E0
    
D = Build_DensMat(C0,s)

with np.printoptions(precision=3, suppress=True):
    print(s)
    
### Stage 1. Solve the Generalized Eigenvalue problem, using S and F, obtaining new C.

small_S, U = np.linalg.eigh(s)
## Step 2: Calculate s^{-1/2}.
half_S = np.zeros((N_tot,N_tot))
for i in range(N_tot):
    half_S[i,i] = 1.0E0/np.sqrt(np.abs(small_S[i]))
    #print(half_S[i,i])
## Step 3: Calculate V = Us^{-1/2}.
V = np.dot(U,half_S)
## Step 4: Calculate H' = V^d H V.
Vd = np.transpose(V)

[[ 1.     0.741  0.426 ...  0.     0.     0.   ]
 [ 0.741  1.     0.819 ...  0.     0.     0.   ]
 [ 0.426  0.819  1.    ...  0.     0.     0.   ]
 ...
 [ 0.     0.     0.    ...  1.     0.    -0.   ]
 [ 0.     0.     0.    ...  0.     1.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     1.   ]]


In [13]:
ener_traj = []
for i in range(20): 
    OldEner = Ener
    G = calc_G(N_tot,D,Q)
    F = calc_F(N_tot,G,Hcore)
    Ener, D_new = DiagFock(s,N_tot,F,Hcore, D,V,Vd,Q,i)
    #print("---D New---")
    #with np.printoptions(precision=3, suppress=True):
    #    print(D_new)
    #plt.imshow(D_new,cmap='Blues')
    #plt.show()
    ### print("Cost: ", np.abs(OldEner - Ener))
    ener_traj.append(Ener)
    D = copy.deepcopy(D_new)

After round    0 Energy:    -2.985539
After round    1 Energy:    -2.831916
After round    2 Energy:    -2.870534
After round    3 Energy:    -2.859026
After round    4 Energy:    -2.862392
After round    5 Energy:    -2.861402
After round    6 Energy:    -2.861693
After round    7 Energy:    -2.861608
After round    8 Energy:    -2.861633
After round    9 Energy:    -2.861625
After round   10 Energy:    -2.861627
After round   11 Energy:    -2.861627
After round   12 Energy:    -2.861627
After round   13 Energy:    -2.861627


KeyboardInterrupt: 